In [1]:
import os
import os.path
import tensorflow as tf
import pickle as pkl
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import plot_model
from keras.models import Model,Sequential
from keras.layers import Input,Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization,Dropout,GlobalAveragePooling2D
from keras.losses import sparse_categorical_crossentropy
from keras.layers.merge import concatenate
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from sklearn.ensemble import VotingClassifier

#from keras.applications.efficientnet import EfficientNetB0

In [2]:
seed_value = 1234
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

#maybe tensorflow and tf differs

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
no_classes = 10
img_width, img_height, img_num_channels = 32, 32, 3

In [5]:
(input_train, target_train), (input_test, target_test) = keras.datasets.cifar10.load_data()

In [6]:
# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
input_train = input_train / 255
input_test = input_test / 255


In [7]:
modelseq = Sequential()
modelseq.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=input_shape))
modelseq.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
modelseq.add(MaxPooling2D((2, 2)))
modelseq.add(Dropout(0.2))
modelseq.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
modelseq.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
modelseq.add(MaxPooling2D((2, 2)))
modelseq.add(Dropout(0.2))
modelseq.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
modelseq.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
modelseq.add(MaxPooling2D((2, 2)))
modelseq.add(Dropout(0.2))
modelseq.add(Flatten())
modelseq.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
modelseq.add(Dropout(0.2))
modelseq.add(Dense(10, activation='softmax'))
# compile model
opt = SGD(lr=0.001, momentum=0.9)
modelseq.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# summarize layers
print(modelseq.summary())
# plot graph
#plot_model(modelseq, to_file='modelseq.png')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [8]:
# first model
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=input_shape))
model2.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))
model2.add(Dropout(0.2))
model2.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model2.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))
model2.add(Dropout(0.2))
model2.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model2.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))
model2.add(Dropout(0.2))
model2.add(Flatten())
model2.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model2.add(Dropout(0.2))
model2.add(Dense(10, activation='softmax'))

# second model
model3 = Sequential()
model3.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=input_shape))
model3.add(BatchNormalization())
model3.add(MaxPooling2D((2, 2)))
model3.add(Dropout(0.2))
model3.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model3.add(BatchNormalization())
model3.add(MaxPooling2D((2, 2)))
model3.add(Dropout(0.2))
model3.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model3.add(BatchNormalization())
model3.add(MaxPooling2D((2, 2)))
model3.add(Dropout(0.2))
model3.add(Flatten())
model3.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model3.add(Dropout(0.2))
model3.add(Dense(10, activation='softmax'))

# third model
model4 = Sequential()
model4.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=input_shape))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))
model4.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))
model4.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))
model4.add(Flatten())
model4.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model4.add(Dropout(0.2))
model4.add(Dense(10, activation='softmax'))


opt = keras.optimizers.Adam(learning_rate=0.001)
opt2 = keras.optimizers.Adam(learning_rate=0.01)
opt3 = SGD(lr=0.001, momentum=0.9)
model2.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model3.compile(optimizer=opt2, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model4.compile(optimizer=opt3, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# summarize layers
print(model2.summary())
print(model3.summary())
print(model4.summary())
# plot graph
#plot_model(model2, to_file='model2.png')
#plot_model(model3, to_file='model3.png')
#plot_model(model4, to_file='model4.png')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 64)       

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [9]:
"""
history = modelseq.fit(input_train, target_train,
            batch_size=64,
            epochs=25,
            verbose=1,
            validation_split=0.2)
"""

'\nhistory = modelseq.fit(input_train, target_train,\n            batch_size=64,\n            epochs=25,\n            verbose=1,\n            validation_split=0.2)\n'

In [10]:
"""
history2 = model2.fit(input_train, target_train,
            batch_size=64,
            epochs=10,
            verbose=1,
            validation_split=0.2)
"""

'\nhistory2 = model2.fit(input_train, target_train,\n            batch_size=64,\n            epochs=10,\n            verbose=1,\n            validation_split=0.2)\n'

In [11]:
"""
history3 = model3.fit(input_train, target_train,
            batch_size=64,
            epochs=10,
            verbose=1,
            validation_split=0.2)
"""

'\nhistory3 = model3.fit(input_train, target_train,\n            batch_size=64,\n            epochs=10,\n            verbose=1,\n            validation_split=0.2)\n'

In [12]:
"""
history4 = model4.fit(input_train, target_train,
            batch_size=64,
            epochs=10,
            verbose=1,
            validation_split=0.2)
"""

'\nhistory4 = model4.fit(input_train, target_train,\n            batch_size=64,\n            epochs=10,\n            verbose=1,\n            validation_split=0.2)\n'

In [13]:
"""
# Generate generalization metrics
score = modelseq.evaluate(input_test, target_test, verbose=0)
score2 = model2.evaluate(input_test, target_test, verbose=0)
score3 = model3.evaluate(input_test, target_test, verbose=0)
score4 = model4.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
print(f'Test loss: {score2[0]} / Test accuracy: {score2[1]}')
print(f'Test loss: {score3[0]} / Test accuracy: {score3[1]}')
print(f'Test loss: {score4[0]} / Test accuracy: {score4[1]}')

# Visualize history
# Plot history: Loss
plt.plot(history.history['accuracy'],label = 'train_1')
plt.plot(history.history['val_accuracy'],label = 'test_1')
plt.plot(history2.history['accuracy'],label = 'train_2')
plt.plot(history2.history['val_accuracy'],label = 'test_2')
plt.plot(history3.history['accuracy'],label = 'train_3')
plt.plot(history3.history['val_accuracy'],label = 'test_3')
plt.plot(history4.history['accuracy'],label = 'train_4')
plt.plot(history4.history['val_accuracy'],label = 'test_4')
plt.legend()
plt.show()
"""

"\n# Generate generalization metrics\nscore = modelseq.evaluate(input_test, target_test, verbose=0)\nscore2 = model2.evaluate(input_test, target_test, verbose=0)\nscore3 = model3.evaluate(input_test, target_test, verbose=0)\nscore4 = model4.evaluate(input_test, target_test, verbose=0)\nprint(f'Test loss: {score[0]} / Test accuracy: {score[1]}')\nprint(f'Test loss: {score2[0]} / Test accuracy: {score2[1]}')\nprint(f'Test loss: {score3[0]} / Test accuracy: {score3[1]}')\nprint(f'Test loss: {score4[0]} / Test accuracy: {score4[1]}')\n\n# Visualize history\n# Plot history: Loss\nplt.plot(history.history['accuracy'],label = 'train_1')\nplt.plot(history.history['val_accuracy'],label = 'test_1')\nplt.plot(history2.history['accuracy'],label = 'train_2')\nplt.plot(history2.history['val_accuracy'],label = 'test_2')\nplt.plot(history3.history['accuracy'],label = 'train_3')\nplt.plot(history3.history['val_accuracy'],label = 'test_3')\nplt.plot(history4.history['accuracy'],label = 'train_4')\nplt.p

In [14]:
"""
#print(input_test[1])
single = np.expand_dims(input_test[250], axis=0)
prediction1 = modelseq.predict(input_test)
prediction2 = model2.predict(input_test)
prediction3 = model3.predict(input_test)
prediction4 = model4.predict(input_test)
"""

'\n#print(input_test[1])\nsingle = np.expand_dims(input_test[250], axis=0)\nprediction1 = modelseq.predict(input_test)\nprediction2 = model2.predict(input_test)\nprediction3 = model3.predict(input_test)\nprediction4 = model4.predict(input_test)\n'

In [15]:
"""
inception_model = EfficientNetB0(include_top=True,input_tensor=Input(shape=(32, 32, 3)))
x = inception_model.output
predictions = Dense(10, activation='softmax')(x)

# i.e. freeze all convolutional InceptionV3 layers
#for layer in inception_model.layers:
#    layer.trainable = False

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=1, patience=5,restore_best_weights=True)
optAdam = keras.optimizers.Adam(learning_rate=0.1)
modified_inc =  Model(inputs=inception_model.input, outputs=predictions)
modified_inc.compile(optimizer=optAdam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#print(input_train.shape)
history_inc = modified_inc.fit(input_train, target_train,
            batch_size=64,
            epochs=100,
            verbose=1,
            validation_split=0.2,
            callbacks=[callback])
"""

"\ninception_model = EfficientNetB0(include_top=True,input_tensor=Input(shape=(32, 32, 3)))\nx = inception_model.output\npredictions = Dense(10, activation='softmax')(x)\n\n# i.e. freeze all convolutional InceptionV3 layers\n#for layer in inception_model.layers:\n#    layer.trainable = False\n\ncallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=1, patience=5,restore_best_weights=True)\noptAdam = keras.optimizers.Adam(learning_rate=0.1)\nmodified_inc =  Model(inputs=inception_model.input, outputs=predictions)\nmodified_inc.compile(optimizer=optAdam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])\n\n#print(input_train.shape)\nhistory_inc = modified_inc.fit(input_train, target_train,\n            batch_size=64,\n            epochs=100,\n            verbose=1,\n            validation_split=0.2,\n            callbacks=[callback])\n"

In [16]:
scores = list()
scores.append(0.8102)
scores.append(0.8131)
scores.append(0.6504)
scores.append(0.6123)


callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=1, patience=5,restore_best_weights=True)

keras_clf = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = lambda:
                            modelseq,
                            batch_size=64,
                            epochs=500,
                            verbose=True,
                            validation_split=0.2,
                            callbacks=[callback])

keras_clf2 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = lambda:
                            model2,
                            batch_size=64,
                            epochs=500,
                            verbose=True,
                            validation_split=0.2,
                            callbacks=[callback])
keras_clf3 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = lambda:
                            model3,
                            batch_size=64,
                            epochs=500,
                            verbose=True,
                            validation_split=0.2,
                            callbacks=[callback])
keras_clf4 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn = lambda:
                            model4,
                            batch_size=64,
                            epochs=500,
                            verbose=True,
                            validation_split=0.2,
                            callbacks=[callback])

keras_clf._estimator_type = "classifier"
keras_clf2._estimator_type = "classifier"
keras_clf3._estimator_type = "classifier"
keras_clf4._estimator_type = "classifier"

models = list()	
models.append(('m1', keras_clf))
models.append(('m2', keras_clf2))
models.append(('m3', keras_clf3))
models.append(('m4', keras_clf4))

# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='soft', weights=scores)
# fit the ensemble on the training dataset
ensemble.fit(input_train,target_train.ravel())
# make predictions on test set
yhat = ensemble.predict(input_test)

Epoch 1/500
625/625 [==============================] - 20s 9ms/step - loss: 2.1559 - accuracy: 0.1920 - val_loss: 1.7112 - val_accuracy: 0.3713
Epoch 2/500
625/625 [==============================] - 5s 8ms/step - loss: 1.7130 - accuracy: 0.3601 - val_loss: 1.5316 - val_accuracy: 0.4518
Epoch 3/500
625/625 [==============================] - 5s 8ms/step - loss: 1.5649 - accuracy: 0.4231 - val_loss: 1.3981 - val_accuracy: 0.5050
Epoch 4/500
625/625 [==============================] - 5s 8ms/step - loss: 1.4534 - accuracy: 0.4702 - val_loss: 1.3293 - val_accuracy: 0.5260
Epoch 5/500
625/625 [==============================] - 5s 8ms/step - loss: 1.3839 - accuracy: 0.5002 - val_loss: 1.2418 - val_accuracy: 0.5653
Epoch 6/500
625/625 [==============================] - 5s 8ms/step - loss: 1.3201 - accuracy: 0.5245 - val_loss: 1.2178 - val_accuracy: 0.5699
Epoch 7/500
625/625 [==============================] - 5s 8ms/step - loss: 1.2559 - accuracy: 0.5459 - val_loss: 1.1359 - val_accuracy: 0.599

In [17]:
# evaluate predictions
score = np.mean(yhat==np.squeeze(target_test))
print(scores)
print('Weighted Avg Accuracy: %.3f' % (score*100))

[0.8102, 0.8131, 0.6504, 0.6123]
Weighted Avg Accuracy: 82.560
